In [1]:
from gpcam import GPOptimizer

import numpy as np
from numpy.random import default_rng
import random

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import os
import csv
import math

In [2]:
def plot(x, y, xlabel = "x", ylabel = 'y'):
    plt.figure(figsize = (20,10))
    plt.plot(x,y)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

def scatter(x,y, xlabel = "x", ylabel = 'y'):
    plt.figure(figsize = (20,10))
    plt.scatter(x,y)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

def plotUQ(x, y, var, xlabel = "x", ylabel = 'y'):
    plt.figure(figsize = (20,10))
    plt.plot(x,y)
    plt.fill_between(x,y - 3. * np.sqrt(var), y + 3. * np.sqrt(var), alpha = 0.5, color = "grey")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

def plot_model(x, y, var , data = (), xlabel = "x", ylabel = 'y'):
    plt.figure(figsize = (20,10))
    plt.plot(x,y)
    plt.scatter(data[0],data[1], color = "black")
    plt.fill_between(x,y - 1. * np.sqrt(var), y + 1. * np.sqrt(var), alpha = 0.5, color = "grey")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.ylim(np.min(y_data),np.max(y_data))
    plt.show()

**Loading the Data**

In [3]:
energy_data_all = np.load("/data/Synthetic Data Generation_1/my_synthetic_energy.npy")
cycle_number = np.load("/data/Synthetic Data Generation_1/my_synthetic_cycleNum.npy")



label_size = 30

num_of_datasets = 50

#considered_batteries = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24])
#considered_batteries = random.sample(range(0,int(1e4)), num_of_datasets)

considered_batteries = ([5546, 9477, 2231, 4437, 7059, 5259, 8330, 1068, 8214, 5888, 3275, 6845, 7671, 
                         299, 5038, 3503, 8673, 2236, 3644, 4980, 993, 7545, 654, 1418, 6090, 7936, 8792, 
                         6910, 2933, 2382, 9730, 8476, 1882, 7986, 7091, 4813, 3086, 3908, 1539, 8567, 2152, 
                         5738, 8646, 9692, 2661, 6766, 7230, 512, 758, 2881])


print(considered_batteries)

plt.figure(figsize = (20,10))
for i in considered_batteries: plt.scatter(cycle_number,energy_data_all[int(i)])

plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
plt.xlabel("Cycle Number",fontsize=label_size)
plt.ylabel("Energy",fontsize=label_size)
plt.show()

#print("max y: ", np.max(energy_data_all))

# Initializing the data to fit the GP model
data_size = num_of_datasets

# All Data
energy_data = energy_data_all[considered_batteries] 


print("energy_data: ", energy_data.shape)

[5546, 9477, 2231, 4437, 7059, 5259, 8330, 1068, 8214, 5888, 3275, 6845, 7671, 299, 5038, 3503, 8673, 2236, 3644, 4980, 993, 7545, 654, 1418, 6090, 7936, 8792, 6910, 2933, 2382, 9730, 8476, 1882, 7986, 7091, 4813, 3086, 3908, 1539, 8567, 2152, 5738, 8646, 9692, 2661, 6766, 7230, 512, 758, 2881]
energy_data:  (50, 50)


**Calculating Variance**

In [4]:
# Checking the variability trend in the data

variances = np.var(energy_data, axis=0)
stds = np.sqrt(variances)
'''
# Plotting the data
plt.figure(figsize = (15,10))
plt.plot(cycle_number,stds, color = "red", linewidth = 3)
plt.xlabel("Cycle Number",fontsize=label_size)
plt.ylabel("Energy Standard Deviation",fontsize=label_size)
plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
plt.show()

'''

x_pred = np.linspace(0,1000,1001).reshape(-1,1)

# Data
x_data = cycle_number.reshape(-1, 1) # repeat cycle 20 times to create x_data
y_data = stds.reshape(-1, 1) # stack energy rows to create y_data


print("x data: ", x_data.shape)
print("y data: ", y_data.shape)


x data:  (50, 1)
y data:  (50, 1)


# Creating the Subfolder in Results

In [ ]:
# Specify the path for the new folder
new_folder_path = f"/results/Methods Figures"

# Create the folder
os.makedirs(new_folder_path, exist_ok=True)

# GP Components

In [5]:
def get_distance_matrix(x1,x2):
    d = np.zeros((len(x1),len(x2)))
    for i in range(x1.shape[1]):
        d += (x1[:,i].reshape(-1, 1) - x2[:,i])**2
    return np.sqrt(d + 1e-16)

# For the Noise
def s(x, my_slope, my_pow, my_intercept):
    o = my_slope * (x**my_pow) + my_intercept
    return o


def my_noise(x,hps,obj):

    my_slope     = hps[2]
    my_pow       = hps[3]
    my_intercept = hps[4]
    
    noise = np.identity(len(x)) * s(x,my_slope,my_pow,my_intercept)

    return noise

def kernel(x1,x2,hps,obj):
    d = get_distance_matrix(x1,x2) 
    k = hps[0] * obj.squared_exponential_kernel(d,hps[1])
    return k



# Mean Function
def mean1(x,hps,obj):
    a = hps[5]
    b = hps[6]
    y = a * x[:,0] + b

    return y

def mean2(x,hps,obj):
    y = hps[2]*np.exp(hps[3]*x[:,0]) + hps[4]
    return y


def mean3(x,hps,obj):

    a = hps[2]
    b = hps[3]
    c = hps[4]

    y = a * x[:,0]**b + c

    return y

# 1 - Linear Model

In [19]:
# Initializing the GP Model
init_hyperparameters = np.array(  [2.27027694e+01, 2.93069911e+02, 1.16049159e-07, 2.65767924e+00,
 1.44496831e-02, 3.34321589e-02, 9.30930652e-01])


my_gp = GPOptimizer(x_data,y_data,
            init_hyperparameters = init_hyperparameters,  # we need enough of those for kernel, noise and prior mean functions
            compute_device='cpu', 
            gp_kernel_function=kernel,
            gp_kernel_function_grad=None, 
            gp_mean_function=mean1, 
            gp_mean_function_grad=None,
            gp_noise_function=my_noise,
            normalize_y=False,
            sparse_mode=False,
            gp2Scale = False,
            store_inv=False, 
            ram_economy=False, 
            args=None)



# Setting the Optimization Bounds for Hyperparameters
bounds = np.empty((7,2))

# Kernel Sq Exp
bounds[0] = np.array([1e-5,6000.])                     # Kernel Variance
bounds[1] = np.array([100.,1000.])                     # Kernel Lengthscale

# Noise
bounds[2] = np.array([1e-10,1.])                       # Noise Slope
bounds[3] = np.array([2,5.])                           # Noise Power
bounds[4] = np.array([0.,6.])                          # Noise Intercept
# Mean
bounds[5] = np.array([1e-5,2.])                      # Mean slope
bounds[6] = np.array([-1.,2.])                       # Mean intercept

#my_gp.train(hyperparameter_bounds=bounds,max_iter=100)



In [8]:
# Plotting the prior
print("hps: ", my_gp.hyperparameters)

trained_hps = np.array(  [2.27027694e+01, 2.93069911e+02, 1.16049159e-07, 2.65767924e+00,
 1.44496831e-02, 3.34321589e-02, 9.30930652e-01])

trained_prior_y_linear = mean1(x_pred,my_gp.hyperparameters,2)
#trained_prior_y = mean1(x_pred,trained_hps,2)

# Plotting the data
plt.figure(figsize = (15,10))
#plt.plot(x_pred,prior_y, color = "blue", linewidth = 3, label = "Prior")
plt.plot(x_data,y_data, color = "red", linewidth = 3, label = "Synthetic Data")
plt.plot(x_pred,trained_prior_y_linear, color = "blue", linewidth = 3, label = "Trained Noise Model")
plt.xlabel("Cycle Number",fontsize=label_size)
plt.ylabel("QoI Standard Deviation",fontsize=label_size)
plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
plt.legend(fontsize=label_size,frameon=False)
#lt.show()

plt.savefig("/results/Methods Figures/Linear Noise Model.png", dpi=600)


hps:  [2.27027694e+01 2.93069911e+02 1.16049159e-07 2.65767924e+00
 1.44496831e-02 3.34321589e-02 9.30930652e-01]


# 2 - Exponential Model

In [18]:
# Initializing the GP Model
init_hyperparameters = np.array([7.67525612e+00,  1.18831225e+02,  2.42683334e+01,  9.99641530e-04,
 -2.53199081e+01])




my_gp1 = GPOptimizer(x_data,y_data,
            init_hyperparameters = init_hyperparameters,  # we need enough of those for kernel, noise and prior mean functions
            compute_device='cpu', 
            gp_kernel_function=kernel,
            gp_kernel_function_grad=None, 
            gp_mean_function=mean2, 
            gp_mean_function_grad=None,
            #gp_noise_function=my_noise,
            normalize_y=False,
            sparse_mode=False,
            gp2Scale = False,
            store_inv=False, 
            ram_economy=False, 
            args=None)

# Setting the Optimization Bounds for Hyperparameters
bounds = np.empty((5,2))

# Kernel Sq Exp
bounds[0] = np.array([1e-5,6000.])                     # Kernel Variance
bounds[1] = np.array([100.,1000.])                     # Kernel Lengthscale

# Noise
#bounds[2] = np.array([1e-10,1.])                       # Noise Slope
#bounds[3] = np.array([1e-5,5.])                           # Noise Power
#bounds[4] = np.array([0.,6.])                          # Noise Intercept
# Mean
bounds[2] = np.array([0.,1e3])                        # Mean slope
bounds[3] = np.array([0.,1e-3])                          
bounds[4] = np.array([-100,100])                          

my_gp1.train(hyperparameter_bounds=bounds,max_iter=100)


array([ 7.72770255e+00,  1.18740292e+02,  2.40998259e+01,  9.99935449e-04,
       -2.47835147e+01])

## Plotting Trained Prior

In [20]:
# Plotting the prior
print("hps: ", my_gp1.hyperparameters)

trained_hps = np.array([7.67525612e+00,  1.18831225e+02,  2.42683334e+01,  9.99641530e-04,
 -2.53199081e+01])

#trained_prior_y = mean2(x_pred,my_gp1.hyperparameters,2)

trained_prior_y_exponential = mean2(x_pred,trained_hps,2)


# Plotting the data
plt.figure(figsize = (15,10))
#plt.plot(x_pred,prior_y, color = "blue", linewidth = 3, label = "Prior")
plt.plot(x_data,y_data, color = "red", linewidth = 3, label = "Synthetic Data")
plt.plot(x_pred,trained_prior_y_exponential, color = "blue", linewidth = 3, label = "Trained Noise Model")
plt.xlabel("Cycle Number",fontsize=label_size)
plt.ylabel("QoI Standard Deviation",fontsize=label_size)
plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
plt.legend(fontsize=label_size,frameon=False)
#plt.show()

plt.savefig("/results/Methods Figures/Exponential Noise Model.png", dpi=600)


hps:  [ 7.72770255e+00  1.18740292e+02  2.40998259e+01  9.99935449e-04
 -2.47835147e+01]


# 3 - Power Law Model

In [21]:
# Initializing the GP Model
init_hyperparameters = np.array( [6.18307113e+00, 1.23421060e+02, 7.57041819e-06, 2.24447839e+00,
 1.99782453e+00])

my_gp2 = GPOptimizer(x_data,y_data,
            init_hyperparameters = init_hyperparameters,  # we need enough of those for kernel, noise and prior mean functions
            compute_device='cpu', 
            gp_kernel_function=kernel,
            gp_kernel_function_grad=None, 
            gp_mean_function=mean3, 
            gp_mean_function_grad=None,
            #gp_noise_function=my_noise,
            normalize_y=False,
            sparse_mode=False,
            gp2Scale = False,
            store_inv=False, 
            ram_economy=False, 
            args=None)

# Setting the Optimization Bounds for Hyperparameters
bounds = np.empty((5,2))

# Kernel Sq Exp
bounds[0] = np.array([1e-5,6000.])                     # Kernel Variance
bounds[1] = np.array([100.,1000.])                     # Kernel Lengthscale

# Noise
#bounds[2] = np.array([1e-10,1.])                       # Noise Slope
#bounds[3] = np.array([1e-5,5.])                           # Noise Power
#bounds[4] = np.array([0.,6.])                          # Noise Intercept
# Mean
bounds[2] = np.array([0,1e-2])                        # Mean slope
bounds[3] = np.array([1.,5.])                          
bounds[4] = np.array([0.,5.])                          

my_gp2.train(hyperparameter_bounds=bounds,max_iter=150)


array([3.19436320e+00, 1.00051499e+02, 4.53671036e-05, 1.97710607e+00,
       2.37397522e+00])

In [22]:
# Plotting the prior
print("hps: ", my_gp2.hyperparameters)

trained_hps = np.array( [6.18307113e+00, 1.23421060e+02, 7.57041819e-06, 2.24447839e+00,
 1.99782453e+00])

trained_prior_y_powerlaw = mean3(x_pred,my_gp2.hyperparameters,2)
#trained_prior_y_powerlaw = mean3(x_pred,trained_hps,2)


# Plotting the data
plt.figure(figsize = (15,10))
#plt.plot(x_pred,prior_y, color = "blue", linewidth = 3, label = "Prior")
plt.plot(x_data,y_data, color = "red", linewidth = 3, label = "Synthetic Data")
plt.plot(x_pred,trained_prior_y_powerlaw, color = "blue", linewidth = 3, label = "Trained Noise Model")
plt.xlabel("Cycle Number",fontsize=label_size)
plt.ylabel("QoI Standard Deviation",fontsize=label_size)
plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
plt.legend(fontsize=label_size,frameon=False)
#plt.show()

plt.savefig("/results/Methods Figures/Power Noise Model.png", dpi=600)


hps:  [3.19436320e+00 1.00051499e+02 4.53671036e-05 1.97710607e+00
 2.37397522e+00]


# Plotting all models together

In [23]:
GT_hps = np.array([0,0,                      # kernel - Not Needed here
                   1.002e-05,  2.69, 3,      # Noise
                500,-0.05,-0.2])             # Mean

GT_stds =  np.sqrt(mean3(cycle_number, GT_hps, 2))


# Plotting the data
plt.figure(figsize = (10,10))
#plt.plot(x_pred,prior_y, color = "blue", linewidth = 3, label = "Prior")
#plt.plot(x_data,y_data, color = "blue", linewidth = 7, linestyle='--', label = "Data")
plt.plot(x_data,GT_stds, color = "blue", linewidth = 7, linestyle='--', label = "Ground Truth Noise")

plt.plot(x_pred,trained_prior_y_linear, color = "green", linewidth = 7, label = "Linear")
plt.plot(x_pred,trained_prior_y_exponential, color = "red", linewidth = 7, label = "Exponential")
plt.plot(x_pred,trained_prior_y_powerlaw, color = "black", linewidth = 7, label = "Power Law")
plt.xlabel("Cycle Number",fontsize=label_size)
plt.ylabel("QoI Standard Deviation",fontsize=label_size)
plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
plt.legend(fontsize=label_size,frameon=False)
plt.xlim([0,1000])
plt.ylim([-1,42])
plt.xticks([0,250,500,750,1000])
plt.yticks([0,10,20,30,40])
#plt.show()
plt.savefig('/results/Methods Figures/All Noise Models.pdf', bbox_inches='tight')
